In [4]:
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
import random
import numpy as np

import warnings
warnings.filterwarnings('ignore')


def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(42)

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
num_classes = 10

# Convert class labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Normalize pixel values between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Define the model architecture
model = keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(num_classes, activation="softmax"))

# Compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Choose hyperparameters
batch_size = 64
epochs = 20

# Start the timer
start_time = time.time()

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Calculate the training time
end_time = time.time()
training_time = end_time - start_time

Epoch 1/20


2023-05-19 12:46:37.621214: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 552960000 exceeds 10% of free system memory.


704/704 [==============================] - 25s 35ms/step - loss: 1.5480 - accuracy: 0.4330 - precision_3: 0.6903 - recall_3: 0.1974 - val_loss: 1.2925 - val_accuracy: 0.5258 - val_precision_3: 0.6941 - val_recall_3: 0.3422
Epoch 2/20
704/704 [==============================] - 25s 35ms/step - loss: 1.2049 - accuracy: 0.5704 - precision_3: 0.7414 - recall_3: 0.3926 - val_loss: 1.1975 - val_accuracy: 0.5880 - val_precision_3: 0.7552 - val_recall_3: 0.3968
Epoch 3/20
704/704 [==============================] - 25s 35ms/step - loss: 1.0622 - accuracy: 0.6240 - precision_3: 0.7657 - recall_3: 0.4815 - val_loss: 1.0224 - val_accuracy: 0.6326 - val_precision_3: 0.7611 - val_recall_3: 0.4996
Epoch 4/20
704/704 [==============================] - 22s 31ms/step - loss: 0.9604 - accuracy: 0.6667 - precision_3: 0.7875 - recall_3: 0.5434 - val_loss: 0.9253 - val_accuracy: 0.6828 - val_precision_3: 0.7990 - val_recall_3: 0.5622
Epoch 5/20
704/704 [==============================] - 20s 28ms/step - loss:

In [17]:
# Evaluate the model on the test set
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(x_test, y_test, verbose=2)
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f"""
accuracy = {test_accuracy:.2f},
precision = {test_precision:.2f},
recall = {test_recall:.2f},
F1 score = {test_f1:.2f}
""")

# Print the training time
print(f'training time: {training_time:.2f} seconds')

313/313 - 1s - loss: 1.0800 - accuracy: 0.7081 - precision_3: 0.7380 - recall_3: 0.6829 - 1s/epoch - 4ms/step

accuracy = 0.71,
precision = 0.74,
recall = 0.68,
F1 score = 0.71

training time: 425.02 seconds
